In [128]:
import pandas as pd
import json as json
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing

In [129]:
df = pd.read_json('preprocessed_business_data.json')
df.head()

,attributes,business_id,checkin_count,city,hours,index,sentiment_value_x,sentiment_value_y,stars,useful
0,"{u'Alcohol': u'full_bar', u'Noise Level': u'av...",mVHrayjG3uZ_RLHkLj-AMg,28,Braddock,"{u'Tuesday': {u'close': u'19:00', u'open': u'1...",0,0.126932,-0.500000,4.5,44.280000
1,"{u'Alcohol': u'full_bar', u'Noise Level': u'lo...",KayYbHCt-RkbGcPdGOThNg,65,Carnegie,"{u'Monday': {u'close': u'02:00', u'open': u'11...",1,0.419308,1.083333,4.0,304.263158
2,"{u'Alcohol': u'none', u'Noise Level': u'averag...",b9WZJp5L1RZr4F1nxclOoQ,125,Carnegie,"{u'Monday': {u'close': u'14:30', u'open': u'06...",2,0.099039,0.518519,4.5,216.609375
3,"{u'Take-out': True, u'Accepts Credit Cards': T...",P1fJb2WQ1mXoiudj8UE44w,95,Carnegie,"{u'Monday': {u'close': u'22:00', u'open': u'11...",3,-0.013215,0.238095,3.5,380.544304
4,"{u'Take-out': True, u'Accepts Credit Cards': T...",6ilJq_05xRgek_8qUp36-g,112,Munhall,"{u'Monday': {u'close': u'00:00', u'open': u'00...",4,-0.368287,0.138889,2.0,355.907407


In [130]:
df_new = df.drop(['attributes','business_id', 'hours', 'index'] , axis=1)

In [131]:
df_new.head()

,checkin_count,city,sentiment_value_x,sentiment_value_y,stars,useful
0,28,Braddock,0.126932,-0.500000,4.5,44.280000
1,65,Carnegie,0.419308,1.083333,4.0,304.263158
2,125,Carnegie,0.099039,0.518519,4.5,216.609375
3,95,Carnegie,-0.013215,0.238095,3.5,380.544304
4,112,Munhall,-0.368287,0.138889,2.0,355.907407


In [132]:
print len(df)

11207


In [133]:
print len(df['city'].unique())

151


In [134]:
cities = df['city'].unique()
le = preprocessing.LabelEncoder()
df_new.city = le.fit_transform(df_new.city)
df_new.head()


,checkin_count,city,sentiment_value_x,sentiment_value_y,stars,useful
0,28,8,0.126932,-0.500000,4.5,44.280000
1,65,13,0.419308,1.083333,4.0,304.263158
2,125,13,0.099039,0.518519,4.5,216.609375
3,95,13,-0.013215,0.238095,3.5,380.544304
4,112,90,-0.368287,0.138889,2.0,355.907407


In [135]:
from sklearn import cross_validation

# Helper function
def folds_to_split(data,targets,train,test):
    data_tr = pd.DataFrame(data).iloc[train]
    data_te = pd.DataFrame(data).iloc[test]
    labels_tr = pd.DataFrame(targets).iloc[train]
    labels_te = pd.DataFrame(targets).iloc[test]
    return [data_tr, data_te, labels_tr, labels_te]

In [136]:
labels = df_new.loc[:,'stars']
df_attrs = df_new.drop(['stars'] , axis=1)
labels.head()


0    4.5
1    4.0
2    4.5
3    3.5
4    2.0
Name: stars, dtype: float64

In [137]:
df_attrs.head()

,checkin_count,city,sentiment_value_x,sentiment_value_y,useful
0,28,8,0.126932,-0.500000,44.280000
1,65,13,0.419308,1.083333,304.263158
2,125,13,0.099039,0.518519,216.609375
3,95,13,-0.013215,0.238095,380.544304
4,112,90,-0.368287,0.138889,355.907407


In [138]:
df_labels = labels.to_frame()
df_labels.head()


,stars
0,4.5
1,4.0
2,4.5
3,3.5
4,2.0


In [139]:
df_attrs.isnull().values.any()

True

In [140]:
df_attrs_clean=df_attrs.fillna(0)

In [141]:
df_attrs_clean.isnull().values.any()

False

In [142]:
# Using Linear Regression
from sklearn import linear_model
regr = linear_model.LinearRegression()
[tr_data, te_data, 
 tr_labels, te_labels] = cross_validation.train_test_split(df_attrs_clean, df_labels, test_size=0.33,random_state=2016)

regr.fit(tr_data, tr_labels)
print("Mean squared error: %.2f"
      % np.mean((regr.predict(te_data) - te_labels) ** 2))
print('Variance score: %.2f' % regr.score(te_data, te_labels))

Mean squared error: 0.16
Variance score: 0.56


In [143]:
df_labels_rounded = df_labels['stars'].round(0)
df_labels_rounded.head()

0    4.0
1    4.0
2    4.0
3    4.0
4    2.0
Name: stars, dtype: float64

In [144]:
gnb = GaussianNB()
[tr_data, te_data, 
 tr_labels, te_labels] = cross_validation.train_test_split(df_attrs_clean, df_labels_rounded, test_size=0.33,random_state=2016)


In [145]:
gnb.fit(tr_data, tr_labels.values.ravel())

GaussianNB()

In [146]:
gnb.score(te_data, te_labels)

0.79535009462016759